In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,14360.95,0.23,0.06,0.04,0.00,-0.00,0.00,-0.00
3,0.23,2791.95,-0.02,0.01,-0.00,0.00,-0.00,0.00
4,0.06,-0.02,451.02,0.00,0.00,-0.00,0.00,-0.00
5,0.04,0.01,0.00,64.13,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,8.42,0.00,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,1.08,-0.00,0.00
8,0.00,-0.00,0.00,0.00,0.00,-0.00,0.16,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.02


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00002,0.00004,0.00000,-0.00000,0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00001,0.00001,-0.00002,0.00003
4,0.00002,-0.00002,1.00000,0.00002,0.00001,-0.00005,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00001,0.00001,-0.00002
6,0.00000,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,-0.00000,0.00001,-0.00005,0.00001,0.00002,1.00000,-0.00003,0.00001
8,0.00000,-0.00002,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.3479658909450101

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.58005668e+07 9.54460119e+05 2.42177190e+04 4.70077999e+02
 7.60002595e+00 1.11779865e-01 1.88474521e-03 1.62488147e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999575,-0.028803,-0.004456,-0.000631,-0.000083,-0.000011,-0.000002,-1.950238e-07
1,0.028666,0.999196,-0.027763,-0.003823,-0.000503,-0.000064,-0.000010,-1.038738e-06
2,0.005228,0.027500,0.999177,-0.029099,-0.003729,-0.000481,-0.000071,-8.514398e-06
3,0.000888,0.004581,0.028836,0.999004,-0.033458,-0.004205,-0.000629,-7.377180e-05
4,0.000146,0.000751,0.004648,0.033115,0.998470,-0.043558,-0.006376,-7.433589e-04
5,0.000025,0.000127,0.000792,0.005554,0.042774,0.996120,-0.076218,-8.599161e-03
6,0.000006,0.000029,0.000176,0.001238,0.009438,0.074352,0.990311,-1.168980e-01
7,0.000001,0.000007,0.000042,0.000293,0.002226,0.017344,0.115904,9.931064e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00042503575238472813,
 0.0008040412974178457,
 0.0008225570761256629,
 0.0009959132473028598,
 0.001529830568478996,
 0.0038796972157881804,
 0.009688980295032956,
 0.0068935853836427086]